<a id='head_selection_3'></a>
# Data handling

In following notebook I'm going to explore data, and transform them to usable version for data visualization (Flight_to_plot table) and Machine Learning (Flight_to_plot_combined). Transformations from this notebook has been implemented in flight_data_transformer function in flight_data_transformation.py file.

Transformed data from this notebook are used in:
* [Flights_delays mashine learning notebook](Flights_delays.ipynb#head_selection_1)
* [Flights_data_visualization notebook](Flights_data_visualization.ipynb#head_selection_2)

In [1]:
import random
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from datetime import date 

In [2]:
data = pd.read_csv('DATA/flights.csv')

C:\Users\damix\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\IPython\core\interactiveshell.py:3062: DtypeWarning: Columns (7,8) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


#### Ref 1) Warning informs that columns 7 and 8 have data types mixed. This is a very important point for the future.

In [3]:
print(data.shape)

(5819079, 31)


In [4]:
data.head()

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
0,2015,1,1,4,AS,98,N407AS,ANC,SEA,5,...,408.0,-22.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,2015,1,1,4,AA,2336,N3KUAA,LAX,PBI,10,...,741.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2,2015,1,1,4,US,840,N171US,SFO,CLT,20,...,811.0,5.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
3,2015,1,1,4,AA,258,N3HYAA,LAX,MIA,20,...,756.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,2015,1,1,4,AS,135,N527AS,SEA,ANC,25,...,259.0,-21.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


# Understanding variables

* *YEAR*, *MONTH*, *DAY*, *DAY_OF_WEEK* - dates of the flight
* AIRLINE  -An identification number assigned by US GOV to identify a unique airline
* FLIGHT_NUMBER -The common unique flight number of the flight.
* TAIL_NUMBER -  The unique aircraft number.
* ORIGIN_AIRPORT, DESTINATION_AIRPORT - code attributed by IATA to identify the airports
* SCHEDULED_DEPARTURE - scheduled time of departure
* SCHEDULED_ARRIVAL - scheduled time of arrival
* DEPARTURE_TIME, ARRIVAL_TIME - real times at which take-off and landing took place
* DEPARTURE_DELAY, ARRIVAL_DELAY - difference (in minutes) between planned and real times
* TAXI_OUT - elapsed time from gate to take-off
* WHEELS_OFF - time of take off
* SCHEDULED_TIME - scheduled duration of flight
* ELAPSED_TIME - time elapsed from taxi out to taxi in
* AIR_TIME - air time of flight
* DISTANCE - distance of flight (in miles),
* WHEELS_ON -  time of landing
* TAXI_IN - time that elapsed from landing and docking in gate
* DIVERTED - 1 if flight have been diverted and landed on another airport that scheduled
* CANCELLED - 1 if flight have been canceleed
* CANCELLATION_REASON - reson of flight cancellation
* AIR_SYSTEM_DELAY, SECURITY_DELAY, AIRLINE_DELAY, LATE_AIRCRAFT_DELAY, WEATHER_DELAY - reasons of delay

***
# Feature engineering

## _na_ handling

In this section I'm handling N/A values. In below result there are shown amount of N/A values in percentage representation.

In [5]:
data.isna().sum()*100/data.shape[0]

YEAR                    0.000000
MONTH                   0.000000
DAY                     0.000000
DAY_OF_WEEK             0.000000
AIRLINE                 0.000000
FLIGHT_NUMBER           0.000000
TAIL_NUMBER             0.252978
ORIGIN_AIRPORT          0.000000
DESTINATION_AIRPORT     0.000000
SCHEDULED_DEPARTURE     0.000000
DEPARTURE_TIME          1.480526
DEPARTURE_DELAY         1.480526
TAXI_OUT                1.530259
WHEELS_OFF              1.530259
SCHEDULED_TIME          0.000103
ELAPSED_TIME            1.805629
AIR_TIME                1.805629
DISTANCE                0.000000
WHEELS_ON               1.589822
TAXI_IN                 1.589822
SCHEDULED_ARRIVAL       0.000000
ARRIVAL_TIME            1.589822
ARRIVAL_DELAY           1.805629
DIVERTED                0.000000
CANCELLED               0.000000
CANCELLATION_REASON    98.455357
AIR_SYSTEM_DELAY       81.724960
SECURITY_DELAY         81.724960
AIRLINE_DELAY          81.724960
LATE_AIRCRAFT_DELAY    81.724960
WEATHER_DE

Above result shows that there are a lot (82%) of missing data in reason of flight delation. Also in cancellation reason there are almost 98%.

In [6]:
data[data['CANCELLED']>0]['ARRIVAL_DELAY'].dropna()

Series([], Name: ARRIVAL_DELAY, dtype: float64)

From the above result, I conclude that there are no delays in the data when the flight is canceled. For this reason, I remove the **CANCELLED** and **CANCELLATION_REASON** columns from the data because in this project I am dealing with delays.

In [7]:
data.isna().sum()

YEAR                         0
MONTH                        0
DAY                          0
DAY_OF_WEEK                  0
AIRLINE                      0
FLIGHT_NUMBER                0
TAIL_NUMBER              14721
ORIGIN_AIRPORT               0
DESTINATION_AIRPORT          0
SCHEDULED_DEPARTURE          0
DEPARTURE_TIME           86153
DEPARTURE_DELAY          86153
TAXI_OUT                 89047
WHEELS_OFF               89047
SCHEDULED_TIME               6
ELAPSED_TIME            105071
AIR_TIME                105071
DISTANCE                     0
WHEELS_ON                92513
TAXI_IN                  92513
SCHEDULED_ARRIVAL            0
ARRIVAL_TIME             92513
ARRIVAL_DELAY           105071
DIVERTED                     0
CANCELLED                    0
CANCELLATION_REASON    5729195
AIR_SYSTEM_DELAY       4755640
SECURITY_DELAY         4755640
AIRLINE_DELAY          4755640
LATE_AIRCRAFT_DELAY    4755640
WEATHER_DELAY          4755640
dtype: int64

Because of that, there is need to drop rows with Cancelled flights.

In [8]:
data.drop(data[(data['CANCELLED']>0)].index, inplace=True)

In [9]:
data.isna().sum()

YEAR                         0
MONTH                        0
DAY                          0
DAY_OF_WEEK                  0
AIRLINE                      0
FLIGHT_NUMBER                0
TAIL_NUMBER                  0
ORIGIN_AIRPORT               0
DESTINATION_AIRPORT          0
SCHEDULED_DEPARTURE          0
DEPARTURE_TIME               0
DEPARTURE_DELAY              0
TAXI_OUT                     0
WHEELS_OFF                   0
SCHEDULED_TIME               1
ELAPSED_TIME             15187
AIR_TIME                 15187
DISTANCE                     0
WHEELS_ON                 2629
TAXI_IN                   2629
SCHEDULED_ARRIVAL            0
ARRIVAL_TIME              2629
ARRIVAL_DELAY            15187
DIVERTED                     0
CANCELLED                    0
CANCELLATION_REASON    5729195
AIR_SYSTEM_DELAY       4665756
SECURITY_DELAY         4665756
AIRLINE_DELAY          4665756
LATE_AIRCRAFT_DELAY    4665756
WEATHER_DELAY          4665756
dtype: int64

In [10]:
data = data.drop(['CANCELLED','CANCELLATION_REASON'], axis=1)

**Diversion**

Flight diversion is a situation when the flight didn't arrived on scheduled arrival airport. Acording to project assumtion (6) I'm not going to handle flight diversion in this project.

I conclude that if the flight was Diverted there won't be any data

In [11]:
data[data['DIVERTED']>0]

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
724,2015,1,1,4,DL,716,N944DL,OMA,ATL,645,...,4.0,1009,1451.0,NaN,1,NaN,NaN,NaN,NaN,NaN
1455,2015,1,1,4,OO,5237,N712SK,MKE,IAH,745,...,9.0,1041,1505.0,NaN,1,NaN,NaN,NaN,NaN,NaN
2218,2015,1,1,4,WN,1966,N685SW,ATL,JAX,845,...,5.0,1000,1219.0,NaN,1,NaN,NaN,NaN,NaN,NaN
2526,2015,1,1,4,EV,4555,N12552,IAH,HRL,902,...,5.0,1017,1319.0,NaN,1,NaN,NaN,NaN,NaN,NaN
2866,2015,1,1,4,WN,1081,N214WN,MDW,OKC,930,...,3.0,1135,1343.0,NaN,1,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5813860,2015,12,31,4,UA,1291,N596UA,LIH,LAX,1405,...,NaN,2141,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN
5815052,2015,12,31,4,EV,2785,N684JW,DFW,BRO,1530,...,NaN,1720,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN
5816657,2015,12,31,4,UA,291,N76505,IAD,SMF,1735,...,4.0,2026,2216.0,NaN,1,NaN,NaN,NaN,NaN,NaN
5817342,2015,12,31,4,MQ,3060,N648MQ,DFW,FAR,1830,...,3.0,2113,2250.0,NaN,1,NaN,NaN,NaN,NaN,NaN


In [12]:
data_without_nulls = data.drop(data[(data['DIVERTED']>0)].index)

In [13]:
data_without_nulls = data_without_nulls.drop(['DIVERTED'], axis=1)

In [14]:
data_without_nulls.isna().sum()

YEAR                         0
MONTH                        0
DAY                          0
DAY_OF_WEEK                  0
AIRLINE                      0
FLIGHT_NUMBER                0
TAIL_NUMBER                  0
ORIGIN_AIRPORT               0
DESTINATION_AIRPORT          0
SCHEDULED_DEPARTURE          0
DEPARTURE_TIME               0
DEPARTURE_DELAY              0
TAXI_OUT                     0
WHEELS_OFF                   0
SCHEDULED_TIME               0
ELAPSED_TIME                 0
AIR_TIME                     0
DISTANCE                     0
WHEELS_ON                    0
TAXI_IN                      0
SCHEDULED_ARRIVAL            0
ARRIVAL_TIME                 0
ARRIVAL_DELAY                0
AIR_SYSTEM_DELAY       4650569
SECURITY_DELAY         4650569
AIRLINE_DELAY          4650569
LATE_AIRCRAFT_DELAY    4650569
WEATHER_DELAY          4650569
dtype: int64

Variables: 'AIR_SYSTEM_DELAY', 'SECURITY_DELAY', 'AIRLINE_DELAY', 'LATE_AIRCRAFT_DELAY', 'WEATHER_DELAY' are indicators of delay reson. 
Knowing those variables would be too obvious for algorithm to define it with occurrence of a delay. Those columns should be dropped.

In [15]:
Flights_Delays_Combined =  data_without_nulls.drop(['AIR_SYSTEM_DELAY','SECURITY_DELAY','AIRLINE_DELAY',
                    'LATE_AIRCRAFT_DELAY','WEATHER_DELAY'],axis = 1)

In [16]:
Flights_Delays_Combined.head()

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY
0,2015,1,1,4,AS,98,N407AS,ANC,SEA,5,...,15.0,205.0,194.0,169.0,1448,404.0,4.0,430,408.0,-22.0
1,2015,1,1,4,AA,2336,N3KUAA,LAX,PBI,10,...,14.0,280.0,279.0,263.0,2330,737.0,4.0,750,741.0,-9.0
2,2015,1,1,4,US,840,N171US,SFO,CLT,20,...,34.0,286.0,293.0,266.0,2296,800.0,11.0,806,811.0,5.0
3,2015,1,1,4,AA,258,N3HYAA,LAX,MIA,20,...,30.0,285.0,281.0,258.0,2342,748.0,8.0,805,756.0,-9.0
4,2015,1,1,4,AS,135,N527AS,SEA,ANC,25,...,35.0,235.0,215.0,199.0,1448,254.0,5.0,320,259.0,-21.0


In [17]:
Flights_Delays_Combined.columns

Index(['YEAR', 'MONTH', 'DAY', 'DAY_OF_WEEK', 'AIRLINE', 'FLIGHT_NUMBER',
       'TAIL_NUMBER', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT',
       'SCHEDULED_DEPARTURE', 'DEPARTURE_TIME', 'DEPARTURE_DELAY', 'TAXI_OUT',
       'WHEELS_OFF', 'SCHEDULED_TIME', 'ELAPSED_TIME', 'AIR_TIME', 'DISTANCE',
       'WHEELS_ON', 'TAXI_IN', 'SCHEDULED_ARRIVAL', 'ARRIVAL_TIME',
       'ARRIVAL_DELAY'],
      dtype='object')

In [18]:
Flights_Delays_Combined.isnull().sum()

YEAR                   0
MONTH                  0
DAY                    0
DAY_OF_WEEK            0
AIRLINE                0
FLIGHT_NUMBER          0
TAIL_NUMBER            0
ORIGIN_AIRPORT         0
DESTINATION_AIRPORT    0
SCHEDULED_DEPARTURE    0
DEPARTURE_TIME         0
DEPARTURE_DELAY        0
TAXI_OUT               0
WHEELS_OFF             0
SCHEDULED_TIME         0
ELAPSED_TIME           0
AIR_TIME               0
DISTANCE               0
WHEELS_ON              0
TAXI_IN                0
SCHEDULED_ARRIVAL      0
ARRIVAL_TIME           0
ARRIVAL_DELAY          0
dtype: int64

There are no missing data left in dataset.

***

## Handling mixed value types

Some airports in following data does not have assigned IATA code. That's why they are represented by number. In this project I'm not going to handle data from small airports that don't have that code.
In refrence to point **1)** I am handling mixed data types by making function that changes numer airport code to unknown ('unk')

In [19]:
Flights_Delays_Combined['ORIGIN_AIRPORT'].value_counts()

ATL      343506
ORD      276554
DFW      232647
DEN      193402
LAX      192003
          ...  
12265         1
14025         1
11097         1
13459         1
11905         1
Name: ORIGIN_AIRPORT, Length: 929, dtype: int64

In [20]:
Flights_Delays_Combined['DESTINATION_AIRPORT'].value_counts()

ATL      343076
ORD      275864
DFW      231764
DEN      193033
LAX      192136
          ...  
13459         1
10666         1
15497         1
12265         1
14025         1
Name: DESTINATION_AIRPORT, Length: 929, dtype: int64

In [21]:
def airport_trainsformation(value):
    '''
    >>> airport_trainsformation('ATL')
    'ATL'
    >>> airport_trainsformation(11097)
    'unk'
    >>> airport_trainsformation(15497)
    'unk'
    '''
    if len(str(value))>3:
        return 'unk'
    else:
        return value

import doctest
doctest.testmod()

TestResults(failed=0, attempted=3)

In [22]:
Flights_Delays_Combined['ORIGIN_AIRPOR'] = Flights_Delays_Combined['ORIGIN_AIRPORT'].apply(airport_trainsformation)
Flights_Delays_Combined['DESTINATION_AIRPORT'] = Flights_Delays_Combined['DESTINATION_AIRPORT'].apply(airport_trainsformation)

In [23]:
Flights_Delays_Combined['DESTINATION_AIRPORT'].value_counts()

unk    482878
ATL    343076
ORD    275864
DFW    231764
DEN    193033
        ...  
DLG        77
STC        77
GST        74
AKN        63
ITH        31
Name: DESTINATION_AIRPORT, Length: 323, dtype: int64

In [24]:
Flights_Delays_Combined = Flights_Delays_Combined.drop(Flights_Delays_Combined[(Flights_Delays_Combined['DESTINATION_AIRPORT']=='unk')].index)

In [25]:
Flights_Delays_Combined = Flights_Delays_Combined.drop(Flights_Delays_Combined[(Flights_Delays_Combined['ORIGIN_AIRPORT']=='unk')].index)

In [26]:
Flights_Delays_Combined['DESTINATION_AIRPORT'].value_counts()

ATL    343076
ORD    275864
DFW    231764
DEN    193033
LAX    192136
        ...  
STC        77
DLG        77
GST        74
AKN        63
ITH        31
Name: DESTINATION_AIRPORT, Length: 322, dtype: int64

In [27]:
#Flights_Delays_Combined = Flights_Delays_Combined.drop(['ORIGIN_AIRPORT','DESTINATION_AIRPORT'], axis=1)

### Not understable data format in time columns
In data dates are in float format. Those should be changed for datetime format. I defined *time_transformation* with doctesting function for that.

In [28]:
Flights_Delays_Combined['DEPARTURE_TIME']

0          2354.0
1             2.0
2            18.0
3            15.0
4            24.0
            ...  
5819074    2355.0
5819075    2355.0
5819076    2350.0
5819077    2353.0
5819078      14.0
Name: DEPARTURE_TIME, Length: 5231130, dtype: float64

In [29]:
max(Flights_Delays_Combined['DEPARTURE_TIME'])

2400.0

Hours are in numerical format. I convert them to datetime

In [30]:
def time_transformation(time):
  """
  >>> time_transformation(0)
  datetime.time(0, 0)
  >>> time_transformation(2400)
  datetime.time(0, 0)
  >>> time_transformation(2359)
  datetime.time(23, 59)
  >>> time_transformation(1200)
  datetime.time(12, 0)
  >>> time_transformation(0734.00)
  datetime.time(7, 34)
  >>> time_transformation(734.00)
  datetime.time(7, 34)
  >>> time_transformation(34.00)
  datetime.time(0, 34)
  >>> time_transformation(4.00)
  datetime.time(0, 4)
  """
  time_string = str(int(time))
  time_string = time_string.split('.')[0]
  time_string = time_string.zfill(4)
  hours = int(time_string[0:2])
  minutes = int(time_string[2:4])
  if hours == 24:
    hours = 0
  hours_minutes = datetime.time(hours,minutes)
  return hours_minutes
  
import doctest
doctest.testmod()

TestResults(failed=0, attempted=11)

In [31]:
Flights_Delays_Combined['DEP_TIME'] = Flights_Delays_Combined['DEPARTURE_TIME'].apply(time_transformation)
Flights_Delays_Combined['DEP_TIME']

0          23:54:00
1          00:02:00
2          00:18:00
3          00:15:00
4          00:24:00
             ...   
5819074    23:55:00
5819075    23:55:00
5819076    23:50:00
5819077    23:53:00
5819078    00:14:00
Name: DEP_TIME, Length: 5231130, dtype: object

In [32]:
Flights_Delays_Combined['SCH_ARR_TIME'] =Flights_Delays_Combined['SCHEDULED_ARRIVAL'].apply(time_transformation)
Flights_Delays_Combined['SCH_DEP_TIME'] =Flights_Delays_Combined['SCHEDULED_DEPARTURE'].apply(time_transformation)
Flights_Delays_Combined['ARR_TIME'] =Flights_Delays_Combined['ARRIVAL_TIME'].apply(time_transformation)
Flights_Delays_Combined['WHEELS_OFF'] =Flights_Delays_Combined['WHEELS_OFF'].apply(time_transformation)
Flights_Delays_Combined['WHEELS_ON'] =Flights_Delays_Combined['WHEELS_ON'].apply(time_transformation)

Combining columns with dates into a single Datetime column

In [34]:
Flights_Delays_Combined['DATE'] = pd.to_datetime(Flights_Delays_Combined[['YEAR','MONTH','DAY']])
Flights_Delays_Combined['DATE']

0         2015-01-01
1         2015-01-01
2         2015-01-01
3         2015-01-01
4         2015-01-01
             ...    
5819074   2015-12-31
5819075   2015-12-31
5819076   2015-12-31
5819077   2015-12-31
5819078   2015-12-31
Name: DATE, Length: 5231130, dtype: datetime64[ns]

In [35]:
Flights_Delays_Combined['DayOfWeek'] = Flights_Delays_Combined['DATE'].apply(date.weekday)
Flights_Delays_Combined = Flights_Delays_Combined.drop(['YEAR','MONTH','DAY','DAY_OF_WEEK'], axis=1)

 Categorical variables such as AIRLINE, PLANE, DEP_ARP, ARR_ARP should also take into account

In [36]:
Flights_Delays_Combined.head()

,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,...,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,ORIGIN_AIRPOR,DEP_TIME,SCH_ARR_TIME,SCH_DEP_TIME,ARR_TIME,DATE,DayOfWeek
0,AS,98,N407AS,ANC,SEA,5,2354.0,-11.0,21.0,00:15:00,...,430,408.0,-22.0,ANC,23:54:00,04:30:00,00:05:00,04:08:00,2015-01-01,3
1,AA,2336,N3KUAA,LAX,PBI,10,2.0,-8.0,12.0,00:14:00,...,750,741.0,-9.0,LAX,00:02:00,07:50:00,00:10:00,07:41:00,2015-01-01,3
2,US,840,N171US,SFO,CLT,20,18.0,-2.0,16.0,00:34:00,...,806,811.0,5.0,SFO,00:18:00,08:06:00,00:20:00,08:11:00,2015-01-01,3
3,AA,258,N3HYAA,LAX,MIA,20,15.0,-5.0,15.0,00:30:00,...,805,756.0,-9.0,LAX,00:15:00,08:05:00,00:20:00,07:56:00,2015-01-01,3
4,AS,135,N527AS,SEA,ANC,25,24.0,-1.0,11.0,00:35:00,...,320,259.0,-21.0,SEA,00:24:00,03:20:00,00:25:00,02:59:00,2015-01-01,3


### Holidays

To get list of holidays I scrapped website www.officeholidays.com which contains dates of official holidays in USA.

In [37]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime
holidays_dates = list()
URL = 'https://www.officeholidays.com/countries/usa/2015'
response = requests.get(URL, timeout=20)
soup = BeautifulSoup(response.text)
for a in soup.find_all('time')[2:]:
    holidays_dates.append(datetime.strptime(a.attrs.get("datetime"),'%Y-%m-%d').date())

In [38]:
holidays_dates

[datetime.date(2015, 2, 16),
 datetime.date(2015, 5, 10),
 datetime.date(2015, 5, 25),
 datetime.date(2015, 6, 21),
 datetime.date(2015, 7, 4),
 datetime.date(2015, 9, 7),
 datetime.date(2015, 10, 12),
 datetime.date(2015, 10, 12),
 datetime.date(2015, 10, 12),
 datetime.date(2015, 11, 26),
 datetime.date(2015, 12, 25)]

In [39]:
Flights_Delays_Combined['Holidays'] = np.where(Flights_Delays_Combined['DATE'].isin(holidays_dates), 1, 0)

In [40]:
Flights_Delays_Combined[Flights_Delays_Combined['Holidays']==1]

,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,...,ARRIVAL_TIME,ARRIVAL_DELAY,ORIGIN_AIRPOR,DEP_TIME,SCH_ARR_TIME,SCH_DEP_TIME,ARR_TIME,DATE,DayOfWeek,Holidays
695436,AA,2400,N5ESAA,LAX,DFW,5,3.0,-2.0,13.0,00:16:00,...,449.0,-4.0,LAX,00:03:00,04:53:00,00:05:00,04:49:00,2015-02-16,0,1
695437,DL,1745,N365NW,SMF,MSP,5,2359.0,-6.0,9.0,00:08:00,...,514.0,-15.0,SMF,23:59:00,05:29:00,00:05:00,05:14:00,2015-02-16,0,1
695438,DL,2579,N693DL,DEN,ATL,15,20.0,5.0,49.0,01:09:00,...,535.0,24.0,DEN,00:20:00,05:11:00,00:15:00,05:35:00,2015-02-16,0,1
695439,US,2020,N917US,PHX,CLT,15,13.0,-2.0,12.0,00:25:00,...,541.0,-19.0,PHX,00:13:00,06:00:00,00:15:00,05:41:00,2015-02-16,0,1
695440,AA,258,N3DEAA,LAX,MIA,20,20.0,0.0,16.0,00:36:00,...,758.0,-6.0,LAX,00:20:00,08:04:00,00:20:00,07:58:00,2015-02-16,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5725388,B6,333,N703JB,MCO,SJU,2359,12.0,13.0,13.0,00:25:00,...,358.0,18.0,MCO,00:12:00,03:40:00,23:59:00,03:58:00,2015-12-25,4,1
5725389,B6,839,N712JB,JFK,BQN,2359,2356.0,-3.0,13.0,00:09:00,...,425.0,-15.0,JFK,23:56:00,04:40:00,23:59:00,04:25:00,2015-12-25,4,1
5725390,B6,1503,N905JB,JFK,SJU,2359,2359.0,0.0,16.0,00:15:00,...,434.0,-6.0,JFK,23:59:00,04:40:00,23:59:00,04:34:00,2015-12-25,4,1
5725391,B6,1248,N907JB,LAS,JFK,2359,19.0,20.0,18.0,00:37:00,...,742.0,1.0,LAS,00:19:00,07:41:00,23:59:00,07:42:00,2015-12-25,4,1


In [41]:
Flights_Delays_Combined.isnull().sum()

AIRLINE                0
FLIGHT_NUMBER          0
TAIL_NUMBER            0
ORIGIN_AIRPORT         0
DESTINATION_AIRPORT    0
SCHEDULED_DEPARTURE    0
DEPARTURE_TIME         0
DEPARTURE_DELAY        0
TAXI_OUT               0
WHEELS_OFF             0
SCHEDULED_TIME         0
ELAPSED_TIME           0
AIR_TIME               0
DISTANCE               0
WHEELS_ON              0
TAXI_IN                0
SCHEDULED_ARRIVAL      0
ARRIVAL_TIME           0
ARRIVAL_DELAY          0
ORIGIN_AIRPOR          0
DEP_TIME               0
SCH_ARR_TIME           0
SCH_DEP_TIME           0
ARR_TIME               0
DATE                   0
DayOfWeek              0
Holidays               0
dtype: int64

In [42]:
Flights_Delays_Combined.head()

,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,...,ARRIVAL_TIME,ARRIVAL_DELAY,ORIGIN_AIRPOR,DEP_TIME,SCH_ARR_TIME,SCH_DEP_TIME,ARR_TIME,DATE,DayOfWeek,Holidays
0,AS,98,N407AS,ANC,SEA,5,2354.0,-11.0,21.0,00:15:00,...,408.0,-22.0,ANC,23:54:00,04:30:00,00:05:00,04:08:00,2015-01-01,3,0
1,AA,2336,N3KUAA,LAX,PBI,10,2.0,-8.0,12.0,00:14:00,...,741.0,-9.0,LAX,00:02:00,07:50:00,00:10:00,07:41:00,2015-01-01,3,0
2,US,840,N171US,SFO,CLT,20,18.0,-2.0,16.0,00:34:00,...,811.0,5.0,SFO,00:18:00,08:06:00,00:20:00,08:11:00,2015-01-01,3,0
3,AA,258,N3HYAA,LAX,MIA,20,15.0,-5.0,15.0,00:30:00,...,756.0,-9.0,LAX,00:15:00,08:05:00,00:20:00,07:56:00,2015-01-01,3,0
4,AS,135,N527AS,SEA,ANC,25,24.0,-1.0,11.0,00:35:00,...,259.0,-21.0,SEA,00:24:00,03:20:00,00:25:00,02:59:00,2015-01-01,3,0


Because DEPARTURE_TIME','SCHEDULED_ARRIVAL', 'SCHEDULED_DEPARTURE', 'ARRIVAL_TIME' have been replaced by datetime represetive (DEP_TIME, 'SCH_ARR_TIME', 'SCH_DEP_TIME', 'ARR_TIME') it need to be dropped.

In [43]:
Flights_Delays_Combined = Flights_Delays_Combined.drop(['DEPARTURE_TIME','SCHEDULED_ARRIVAL', 'SCHEDULED_DEPARTURE', 'ARRIVAL_TIME'], axis=1)

That transformed DataFrame is used in following notebooks:
* [Flights_delays mashine learning notebook](Flights_delays.ipynb#head_selection_1)
* [Flights_data_visualization notebook](Flights_data_visualization.ipynb#head_selection_2)
